In [ ]:
### LDA

In [217]:
import codecs
import re

### APPRENDRE

#Prepare training data
fname = './president/corpus.tache1.learn.utf8'
f=codecs.open(fname, 'r','utf-8') # pour régler le codage
txts=f.readlines()
nblignes=len(txts)

#cptM= 0
#cptC=0
labels = [] # M:-1, C:1
trainData = []
for t in txts:
    label = re.sub(r"<[0-9]*:[0-9]*:(.)>.*","\\1",t)
    txt = re.sub(r"<[0-9]*:[0-9]*:.> (.*)","\\1",t)
    
    trainData.append(txt)   
    
    if label[0]=="C":
        #cptC+=1
        labels.append(1)
    else:
        #cptM +=1
        labels.append(-1)
    
f.close()

In [218]:
from nltk.corpus import stopwords
import nltk
import spacy

#Pre-processing
def clean_text(textData,labels):
    list_stopwords = stopwords.words('french')
    list_stopwords.extend(['quand','avoir','être','là','où','à','donc','aucun','aucune','car','a','ici','plus','comment','abord','toujours','très','celui','celle','celles','encore','autre']) #merci?
    list_stopwords.extend(['bien','vers','déjà','aussi','devoir','falloir','monsieur','madame','messieurs','mesdames','mademoiselle','mesdemoiselles','tout','faire','pouvoir','aussi','comme','dire','al','entre','sans','aujourdhui','si','cela','deux','dont','aller','chacun','chacune','an','depuis','venir','beaucoup','parce','notamment','quelque','chaque','contre','ainsi','voir','lequel','laquelle','lesquels','lesquelles','apresmidi','vouloir'])
    #print(list_stopwords)
    
    #stemmer = nltk.stem.snowball.FrenchStemmer()
    nlp = spacy.load('fr_core_news_sm')
    
    trainC=[]
    trainM=[]
    for i in range(len(textData)):
        text= re.sub(r"\n", " ", textData[i])  #remove "\n"
        #text= re.sub(r"'", " ", text)  #remove '
        
        doc=nlp(text)
        text=' '.join(token.lemma_ for token in doc)
        
        pure_text = ''
        for letter in text:
            if letter.isalpha() or letter==' ': #remove punctuation
                pure_text += letter.lower() #lower case     
        #text = ' '.join(stemmer.stem(word) for word in pure_text.split() if word not in list_stopwords) #racinisation   
        text = [word for word in pure_text.split() if word not in list_stopwords] #remove stopwords
             
        textData[i]=text
        if labels[i]==1: #C
            trainC.append(text)
        else:
            trainM.append(text)
        
    return textData,trainC,trainM


train_pure,trainC,trainM=clean_text(trainData,labels)

In [ ]:
from gensim import corpora,models

dictionary= corpora.Dictionary(train_pure)
corpus= [dictionary.doc2bow(text) for text in train_pure]

In [239]:
from gensim import corpora,models

dictionary_all = corpora.Dictionary(train_pure)
corpus_all = [dictionary_all.doc2bow(text) for text in train_pure]
#print(dictionary.token2id)
#print(corpus)
dictionaryC = corpora.Dictionary(trainC)
corpusC = [dictionaryC.doc2bow(text) for text in trainC]
dictionaryM = corpora.Dictionary(trainM)
corpusM = [dictionaryM.doc2bow(text) for text in trainM]


lda_all = gensim.models.ldamodel.LdaModel(corpus=corpus_all, id2word=dictionary_all, num_topics=10)
topics_all=lda_all.print_topics(num_topics=10, num_words=30)
ldaC = gensim.models.ldamodel.LdaModel(corpus=corpusC, id2word=dictionaryC, num_topics=10)
topicsC=ldaC.print_topics(num_topics=10, num_words=30)
ldaM = gensim.models.ldamodel.LdaModel(corpus=corpusM, id2word=dictionaryM, num_topics=10)
topicsM=ldaM.print_topics(num_topics=10, num_words=30)

In [257]:
def extract_topics(topics):
    res=[]
    for t in topics:
        dict_tmp={}
        tmp=t[1].split(" + ")
        tmp_splited=[r.split('*') for r in tmp]
        for ts in tmp_splited:
            dict_tmp[ts[1][1:-1]]=ts[0]
        res.append(dict_tmp)
    return res
    
dictTopics_all=extract_topics(topics_all)
dictTopicsC=extract_topics(topicsC)
dictTopicsM=extract_topics(topicsM)

In [280]:
def normalize_freq(dictTopics):
    norm=[]
    for d in dictTopics:
        normFreq={}
        somme=sum(float(v) for v in d.values())
        for k in d.keys():
            normFreq[k]=float(d[k])/somme
        norm.append(normFreq)
    return norm

norm_dict_all=normalize_freq(dictTopics_all)
norm_dictC=normalize_freq(dictTopicsC)
norm_dictM=normalize_freq(dictTopicsM)

In [364]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def draw_wordCloud(dicts,name):
    for i in range(len(dicts)):        
        wc = WordCloud(background_color='white',width=400,height=300,max_font_size=50,min_font_size=10,mode='RGBA')
        if len(dicts[i])>0:
            wc.generate_from_frequencies(dicts[i])
        else:
            wc.generate_from_frequencies({"NONE":1})
        filename="./img/"+name+"/topic"+str(i)+".png"
        wc.to_file(filename)
        """
        plt.imshow(wc)
        plt.axis("off")
        plt.show()
        """
        
draw_wordCloud(norm_dict_all,"withFunctionWord/all_10topics_30words")
draw_wordCloud(norm_dictC,"withFunctionWord/C_10topics_30words")
draw_wordCloud(norm_dictM,"withFunctionWord/M_10topics_30words")

In [346]:
def remove_functionWord(dictTopics,lda,dictionary,probaMin,limit):
    res=[]
    for d in dictTopics:
        dic={}
        for k in d.keys():
            if len(lda.get_term_topics(dictionary.token2id[k],minimum_probability=probaMin))<limit:
                dic[k]=d[k]
        res.append(dic)    
    return res

In [365]:
#supprimer les mots qui apparaissent avec une forte proba(0.002) dans la plupart des clusters(ici >=3)
rmvFW_all=remove_functionWord(dictTopics_all,lda_all,dictionary_all,0.001,3)
rmvFW_C=remove_functionWord(dictTopicsC,ldaC,dictionaryC,0.001,3)
rmvFW_M=remove_functionWord(dictTopicsM,ldaM,dictionaryM,0.001,3)

rmvFW_norm_dict_all=normalize_freq(rmvFW_all)
rmvFW_norm_dictC=normalize_freq(rmvFW_C)
rmvFW_norm_dictM=normalize_freq(rmvFW_M)

draw_wordCloud(rmvFW_norm_dict_all,"rmvFunctionWord/all_10topics_30words")
draw_wordCloud(rmvFW_norm_dictC,"rmvFunctionWord/C_10topics_30words")
draw_wordCloud(rmvFW_norm_dictM,"rmvFunctionWord/M_10topics_30words")

In [366]:
#chercher les mots discriminants dans un cluster mais absent dans un autre

#avec 30mots, possible d'avoir des ensembles vide
#[{'cancer': '0.006', 'élection': '0.005', 'municipal': '0.005'}, {}, {}, {}, {'prix': '0.007'}, {}, {}, {}, {'dix': '0.006'}, {}]
#donc, on a besoin de prendre plus de mots
more_topics_all=lda_all.print_topics(num_topics=10, num_words=50)
more_topicsC=ldaC.print_topics(num_topics=10, num_words=50)
more_topicsM=ldaM.print_topics(num_topics=10, num_words=50)

more_dictTopics_all=extract_topics(more_topics_all)
more_dictTopicsC=extract_topics(more_topicsC)
more_dictTopicsM=extract_topics(more_topicsM)

discriminant_all=remove_functionWord(more_dictTopics_all,lda_all,dictionary_all,0,2)
discriminantC=remove_functionWord(more_dictTopicsC,ldaC,dictionaryC,0,2)
discriminantM=remove_functionWord(more_dictTopicsM,ldaM,dictionaryM,0,2)

norm_discriminant_all=normalize_freq(discriminant_all)
norm_discriminantC=normalize_freq(discriminantC)
norm_discriminantM=normalize_freq(discriminantM)

draw_wordCloud(norm_discriminant_all,"discriminant/all_10topics_50words")
draw_wordCloud(norm_discriminantC,"discriminant/C_10topics_50words")
draw_wordCloud(norm_discriminantM,"discriminant/M_10topics_50words")

In [ ]:
#############################################################################################################################################

In [ ]:
### Word2Vec

In [186]:
from gensim.models import word2vec

#Prepare training data
word2_data=[]
for i in range(nblignes):
    if labels[i]==1: # M:-1, C:1
        word2_data.append("Chirac: "+trainData[i])
    else:
        word2_data.append("Mitterrand: "+trainData[i])

In [187]:
#from nltk.corpus import stopwords
import nltk
import spacy

#Pre-processing
def word2_clean_text(textData):
    nlp = spacy.load('fr_core_news_sm')
    
    for i in range(len(textData)):
        text= re.sub(r"\n", " ", textData[i])  #remove "\n"
        
        doc=nlp(text)
        text=' '.join(token.lemma_ for token in doc) #lemmatization
        
        pure_text = ''
        for letter in text:
            if letter.isalpha() or letter==' ': #remove punctuation
                pure_text += letter.lower() #lower case     
        text = [word for word in pure_text.split()]
             
        textData[i]=text
        
    return textData

word2_pure=word2_clean_text(word2_data)

In [190]:
model = word2vec.Word2Vec(word2_pure,sg=1,size=100,window=5,min_count=5,workers=4)
model.save("president_model")

In [211]:
#TEST
model.wv.similar_by_word('chirac', topn =50)

[('mitterrand', 0.7210680842399597),
 ('collectivement', 0.6074990034103394),
 ('auxquels', 0.603534460067749),
 ('persuadé', 0.5978818535804749),
 ('vigiler', 0.5830539464950562),
 ('suggérer', 0.5809817314147949),
 ('chicago', 0.5804547667503357),
 ('souhaitais', 0.5804014205932617),
 ('démonstration', 0.5801637768745422),
 ('accompli', 0.5794323682785034),
 ('réussi', 0.5785024762153625),
 ('convaincr', 0.5783129930496216),
 ('magnifier', 0.5763499140739441),
 ('redi', 0.576248824596405),
 ('termine', 0.5754048824310303),
 ('nigeria', 0.5749176144599915),
 ('pluriel', 0.5722047686576843),
 ('reconnaisser', 0.570411205291748),
 ('néanmoins', 0.5693356990814209),
 ('initiateur', 0.568634033203125),
 ('pardonner', 0.5686154365539551),
 ('incarnez', 0.5678385496139526),
 ('glasgow', 0.5673726201057434),
 ('convaincu', 0.5672576427459717),
 ('retrouvon', 0.5667624473571777),
 ('parachever', 0.5662547945976257),
 ('que', 0.5657082200050354),
 ('habité', 0.5651386380195618),
 ('chinon', 0.

In [210]:
model.wv.similar_by_word('mitterrand', topn =50)

[('chirac', 0.7210680246353149),
 ('pardonner', 0.6413301229476929),
 ('redi', 0.6142987608909607),
 ('cognac', 0.6086660623550415),
 ('tromper', 0.6048446297645569),
 ('insiste', 0.6042050719261169),
 ('reconnais', 0.6024350523948669),
 ('connai', 0.5973896980285645),
 ('aussitôt', 0.5968186855316162),
 ('pourrai', 0.5966140627861023),
 ('excuser', 0.5958927273750305),
 ('trancher', 0.5958324074745178),
 ('attarder', 0.5943892002105713),
 ('quelqu', 0.5934343934059143),
 ('confidence', 0.5907344818115234),
 ('tort', 0.5905795693397522),
 ('chinon', 0.5904669761657715),
 ('dirai', 0.5900932550430298),
 ('fût', 0.5880401134490967),
 ('regret', 0.5878316164016724),
 ('volontiers', 0.5865918397903442),
 ('répéter', 0.586157500743866),
 ('pardonnez', 0.5858516693115234),
 ('miracle', 0.5856518745422363),
 ('ressembler', 0.5842971801757812),
 ('excuse', 0.5841436386108398),
 ('fortiori', 0.5821635723114014),
 ('quoi', 0.5820456147193909),
 ('retenir', 0.5814182758331299),
 ('regretter', 0.5

In [209]:
model.wv.most_similar(positive=["chirac"],negative=["mitterrand"],topn=20)

[('tunisie', 0.3573126792907715),
 ('solidarité', 0.3236124515533447),
 ('inlassable', 0.3187448978424072),
 ('renouveler', 0.3100992739200592),
 ('participe', 0.30520692467689514),
 ('universalité', 0.3013571500778198),
 ('action', 0.30108070373535156),
 ('neutralité', 0.3000565767288208),
 ('encouragement', 0.2965849041938782),
 ('redonner', 0.2915741205215454),
 ('association', 0.2900245189666748),
 ('renouveau', 0.2879122495651245),
 ('édification', 0.2877460718154907),
 ('efficacité', 0.28675806522369385),
 ('excellence', 0.28661903738975525),
 ('enracinement', 0.2854979634284973),
 ('priorité', 0.28395313024520874),
 ('renforcemer', 0.2839243710041046),
 ('solidité', 0.2836388945579529),
 ('pacte', 0.2827143371105194)]

In [213]:
model.wv.most_similar(positive=["mitterrand"],negative=["chirac"],topn=20)

[('on', 0.27397143840789795),
 ('répéter', 0.246195986866951),
 ('lire', 0.24089786410331726),
 ('arrive', 0.2374349981546402),
 ('arriver', 0.2310277223587036),
 ('chose', 0.22063590586185455),
 ('discours', 0.21441566944122314),
 ('apercevoir', 0.21405036747455597),
 ('acheter', 0.2096652090549469),
 ('assez', 0.20324957370758057),
 ('pire', 0.20241950452327728),
 ('durer', 0.20227405428886414),
 ('douze', 0.20203140377998352),
 ('aussitôt', 0.19896963238716125),
 ('séparer', 0.19762855768203735),
 ('sinon', 0.19760464131832123),
 ('gens', 0.19339323043823242),
 ('dire', 0.1918984055519104),
 ('faudrait', 0.18919777870178223),
 ('finalement', 0.18886950612068176)]